In [1]:
!python -m pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import mysql.connector
import pickle
from sklearn.linear_model import LogisticRegression


conn_config = {
    'host': '192.168.1.185',  # our group's central auth ip address
    'user': 'clientUser',
    'password': 'ronaldo7',
    'database': 'fl_database'
}
conn = mysql.connector.connect(**conn_config)
cursor = conn.cursor()


cursor.execute(
    "SELECT model_blob FROM central_updates WHERE model_id = %s ORDER BY version DESC LIMIT 1",
    (1,)
)

result = cursor.fetchone()
cursor.close()
conn.close()

if result is None:
    raise Exception("Model not found in database.")
model_blob = result[0]
model_blob

In [ ]:
if isinstance(model_blob, str):
    model_blob = model_blob.encode('latin1')  # Or 'utf-8' if needed
model = pickle.loads(model_blob)
model

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
data = pd.read_csv('../client_dataset/client_2_binary.csv')

In [3]:
data.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,2.0,0.0,0.0,10.0,6.0,8.0
1,1.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,2.0,10.0,0.0,1.0,0.0,6.0,6.0,4.0
2,0.0,1.0,1.0,1.0,31.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,4.0,0.0,0.0,0.0,1.0,9.0,4.0,8.0
3,0.0,0.0,1.0,1.0,23.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,7.0,6.0,8.0
4,0.0,1.0,1.0,1.0,35.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,5.0,0.0,0.0,0.0,7.0,6.0,8.0


In [4]:
X = data.drop('Diabetes_binary', axis=1)
y = data['Diabetes_binary']

In [5]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [32]:
! pip install optuna 


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [33]:
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('../client_dataset/client_2_binary.csv')
X = data.drop("Diabetes_binary", axis=1)
y = data["Diabetes_binary"]

In [36]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, f1_score, classification_report, accuracy_score
import numpy as np

# Step 1: Identify the minority class from the training data
from collections import Counter
minority_class = min(Counter(y_train), key=Counter(y_train).get)

def objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs', 'newton-cg'])

    if penalty == 'l1' and solver not in ['liblinear', 'saga']:
        raise optuna.exceptions.TrialPruned()
    if penalty == 'l2' and solver not in ['liblinear', 'saga', 'lbfgs', 'newton-cg']:
        raise optuna.exceptions.TrialPruned()

    C = trial.suggest_float('C', 1e-3, 10, log=True)
    max_iter = trial.suggest_int('max_iter', 100, 300)
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])

    model = LogisticRegression(
        penalty=penalty,
        C=C,
        solver=solver,
        max_iter=max_iter,
        class_weight=class_weight,
        random_state=42,
        n_jobs=-1,
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    
    recall_min = recall_score(y_test, y_pred, pos_label=minority_class)
    f1_min = f1_score(y_test, y_pred, pos_label=minority_class)

    
    return recall_min + 0.5 * f1_min  


In [38]:
# Run Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best parameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2025-07-26 00:42:27,680] A new study created in memory with name: no-name-3a9fe788-c0fb-4740-8740-20ac850f2fa3
[I 2025-07-26 00:42:27,683] Trial 0 pruned. 
[I 2025-07-26 00:42:27,683] Trial 1 pruned. 
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2025-07-26 00:42:28,458] Trial 2 finished with value: 0.14606767904012 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.003251334907503341, 'max_iter': 126, 'class_weight': None}. Best is trial 2 with value: 0.14606767904012.
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-07-26 00:42:31,972] Trial 3 finished with value: 0.9951865890916588 and parameters: {'penalty': '

Best hyperparameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.027815504293823668, 'max_iter': 123, 'class_weight': 'balanced'}


In [39]:


# Retrain with best model
best_model = LogisticRegression(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train, y_train)

# Final predictions and evaluation
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


Accuracy: 0.7269000315357931
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.72      0.82     10906
         1.0       0.31      0.78      0.44      1778

    accuracy                           0.73     12684
   macro avg       0.63      0.75      0.63     12684
weighted avg       0.86      0.73      0.77     12684



In [ ]:
import mysql.connector
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, log_loss


accuracy = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, y_pred)


model_blob = pickle.dumps(best_model)


report = classification_report(y_test, y_pred, output_dict=True)


macro_f1 = report["macro avg"]["f1-score"]
recall_minority = report["1.0"]["recall"]
f1_minority = report["1.0"]["f1-score"]
f1_majority = report["0.0"]["f1-score"]


fit_status = "good"


model_id = 1
client_id = "2"
round_num = 2


conn = mysql.connector.connect(
    host= "192.168.1.185",  # our group's central auth ip address
    user="clientUsers",
    password="ronaldo7",
    database="fl_database"
)
cursor = conn.cursor()


insert_query = """
INSERT INTO client_updates 
(model_id, client_id, model_blob, accuracy, loss, round_num, macro_f1, recall_minority, f1_minority, f1_majority, fit_status)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


cursor.execute(insert_query, (
    model_id, client_id, model_blob, accuracy, loss, round_num,
    macro_f1, recall_minority, f1_minority, f1_majority, fit_status
))
conn.commit()

print(" Model update inserted successfully with classification metrics.")

cursor.close()
conn.close()


 Model update inserted successfully with classification metrics.
